# Churn Prediction with Amazon SageMaker Autopilot
---

Amazon SageMaker Autopilot은 테이블 형식의 정형(tabular) 데이터셋을 위한 자동화된 머신 러닝 (일반적으로 AutoML 이라고 합니다) 솔루션입니다. 
현재 Amazon SageMaker Autopilot은 아래 기능들을 지원합니다.
* 테이블 형식의 입력 데이터; 자동 데이터 정제 및 전처리 지원
* 선형 회귀(Linear Regression), 이진 분류(Binary Classification), 및 다중 클래스(Multi-class Classification) 분류를 위한 자동 알고리즘 선택
* 자동 하이퍼 파라미터 최적화(Automatic Hyperparameter Optimization)
* 분산 학습(Distributed Training)
* 인스턴스 및 클러스터 크기 선택 자동 수행

이 노트북은 Churn Prediction 데이터셋에서 Autopilot을 사용하여 여러 가지 잠재적인 옵션 또는
"후보자(candidates)"를 탐색하여 가장 정확한 ML 파이프라인을 얻는 방법을 보여줍니다. Amazon SageMaker Autopilot을 사용하기 위한 준비 작업은 매우 간단합니다.

* CSV/txt 포맷의 정형 데이터를 다운로드받은 후, 학습셋/검증셋으로 분리하여 Amazon S3에 업로드합니다. 이 작업을 본 jupyter 노트북에서 수행합니다.
* Amazon SageMaker Studio나 AWS SDK/CLI를 사용하여 Autopilot을 사용합니다. Autopilot에 의해 생성된 각 후보는 두 단계로 구성됩니다. 본 핸즈온에서는 Amazon SageMaker Studio로 이를 수행해 봅니다.
    * Step 1. 데이터셋에서 자동화된 피쳐 엔지니어링 수행
    * Step 2. 모델 학습 및 하이퍼파라메터 최적화
    
Churn Prediction 데이터셋에 대한 자세한 내용은 아래의 논문을 참조하십시오.
* Paper: https://erpublication.org/published_paper/IJETR032129.pdf
* Dataset URL: http://dataminingconsultant.com/DKD2e_data_sets.zip

## 준비

In [1]:
%%sh
pip install -q --upgrade pip
pip install -q sagemaker smdebug smdebug-rulesconfig==0.1.2 awscli pandas --upgrade

In [2]:
import sagemaker
import pandas as pd
import os
os.chdir("/")
wk_folder = "root/sagemaker-studio-workshop-kr/autopilot/"
os.chdir(wk_folder)

print("Working Folder: ", os.getcwd())

Working Folder:  /root/sagemaker-studio-workshop-kr/autopilot


## 데이터셋 로드

In [3]:
import pandas as pd

df = pd.read_csv('churn.txt')
num_samples = len(df)
pd.set_option('display.max_rows', 50)
df.shape

(3333, 21)

In [4]:
df['Churn?'].value_counts()

False.    2850
True.      483
Name: Churn?, dtype: int64

In [5]:
df.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False.
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False.
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False.
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.


## 데이터셋 분할 후 Amazon S3에 각 데이터셋 업로드

In [6]:
import numpy as np
train_df, valid_df ,_ = np.split(df.sample(frac=1, random_state=42),
                               [int(0.8 * num_samples), int(num_samples)])

In [7]:
train_df.to_csv('automl-train.csv', index=False, header=True, sep=',')
valid_df.to_csv('automl-valid.csv', index=False, header=True, sep=',')

In [8]:
import sagemaker
bucket = sagemaker.Session().default_bucket() # 기존에 버킷이 있다면 기존 버킷 이름을 넣으세요
# bucket = 'sagemaker-studio-gonsoomoon' 
prefix = 'automl-house-price/input'
sess = sagemaker.Session()
uri = sess.upload_data(path='automl-train.csv', bucket = bucket, key_prefix=prefix)

s3_train_path = "s3://" + os.path.join(bucket,prefix) + "/"
print("S3 location: \n", s3_train_path)

'upload_data' method will be deprecated in favor of 'S3Uploader' class (https://sagemaker.readthedocs.io/en/stable/s3.html#sagemaker.s3.S3Uploader) in SageMaker Python SDK v2.


S3 location: 
 s3://sagemaker-us-east-2-101606582575/automl-house-price/input/


In [9]:
! aws s3 ls {s3_train_path}

2020-06-08 05:26:34     250596 automl-train.csv


## QuickSight에서 사용할 Test 파일 생성

In [10]:
def save_test_df(valid_df, test_file_name):
    """
    Drop column, Churn? 
    """
    
    test_df = valid_df.copy()
    test_df.drop(['Churn?'], axis=1, inplace=True)

    test_df.to_csv(test_file_name, index=False, header=True, sep=',')    
    print('{} is saved'.format(test_file_name))
    
    
    return test_df

test_file_name = 'automl-test.csv'
test_df = save_test_df(valid_df, test_file_name)
test_df.head(2)

automl-test.csv is saved


,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls
823,PA,77,510,346-6941,no,yes,24,149.4,74,25.40,123.9,72,10.53,174.3,84,7.84,10.1,6,2.73,1
1531,DE,138,510,380-7816,yes,no,0,46.5,104,7.91,186.0,114,15.81,167.5,95,7.54,9.6,4,2.59,4


In [11]:
prefix = 'autopilot-churn/test'
uri = sess.upload_data(path=test_file_name, bucket = bucket, key_prefix=prefix)

s3_test_path = "s3://" + os.path.join(bucket,prefix) + "/"
print("S3 location: \n", s3_test_path)

'upload_data' method will be deprecated in favor of 'S3Uploader' class (https://sagemaker.readthedocs.io/en/stable/s3.html#sagemaker.s3.S3Uploader) in SageMaker Python SDK v2.


S3 location: 
 s3://sagemaker-us-east-2-101606582575/autopilot-churn/test/


In [12]:
! aws s3 ls {s3_test_path}

2020-06-08 05:26:35      58281 automl-test.csv
